Импортируем необходимые библиотеки

In [34]:
import pandas as pd

Загружаем данные из папки data

In [35]:
train_data = pd.read_csv('data/train.csv')
lectures_data = pd.read_csv('data/lectures.csv')
question_data = pd.read_csv('data/questions.csv')

Выводим первые 5 строк каждого датафрейма

In [36]:
display(train_data.head())
display(lectures_data.head())
display(question_data.head())

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


Выводим общую информацию о датафреймах 

In [37]:

lectures_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          999999 non-null  int64  
 1   timestamp                       999999 non-null  int64  
 2   user_id                         999999 non-null  int64  
 3   content_id                      999999 non-null  int64  
 4   content_type_id                 999999 non-null  int64  
 5   task_container_id               999999 non-null  int64  
 6   user_answer                     999999 non-null  int64  
 7   answered_correctly              999999 non-null  int64  
 8   prior_question_elapsed_time     976276 non-null  float64
 9   prior_question_had_explanation  996183 non-null  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 76.3+ MB


In [39]:
question_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


Получим несколько интересных для исследования метрик

Процент правильно отвеченных вопросов:

In [40]:
correct_answers_count = (train_data['answered_correctly'] == 1).sum()
correct_answers_count / train_data.shape[0] * 100

63.741163741163746

Среднее время, затраченное на ответ

In [41]:
train_data['prior_question_elapsed_time'].mean()

25302.941492979444

Влияние объяснений на результат

In [42]:
# Разделение данных на две группы
group_with_explanation = train_data[train_data['prior_question_had_explanation'] == True]
group_without_explanation = train_data[train_data['prior_question_had_explanation'] == False]

# Подсчет процента правильных ответов в каждой группе
percentage_correct_with_explanation = (group_with_explanation['answered_correctly'] == 1).sum() / group_with_explanation.shape[0] * 100
percentage_correct_without_explanation = (group_without_explanation['answered_correctly'] == 1).sum() / group_without_explanation.shape[0] * 100
display(percentage_correct_with_explanation)
display(percentage_correct_without_explanation)

66.59133866585158

39.725486137896645

Исследуем также файл с вопросами
Например, проанализируем как тэги влияют на правильность ответов


In [43]:
student_questions_df = train_data[train_data['content_type_id'] == 0]

# Объединить DataFrame'ы по content_id (студент) и question_id (вопросы)
merged_df = pd.merge(student_questions_df, question_data, 
                     left_on='content_id', right_on='question_id', 
                     how='inner')
# Проверить, является ли тип данных tags строковым с запятыми
if merged_df['tags'].dtype == 'object':
    # Разделить метки на списки
    merged_df['tags'] = merged_df['tags'].str.split()
    
# Развернуть столбец tags, чтобы иметь одну метку на строку
merged_df = merged_df.explode('tags')
# Вычислить среднюю правильность ответов по каждой метке
tag_performance = merged_df.groupby('tags')['answered_correctly'].mean().reset_index()

# Отсортировать метки по правильности ответов
tag_performance_sorted = tag_performance.sort_values(by='answered_correctly', ascending=False)
# Вычислить количество ответов по каждой метке
tag_counts = merged_df.groupby('tags')['answered_correctly'].count().reset_index()
tag_counts.rename(columns={'answered_correctly': 'count'}, inplace=True)

# Объединить производительность и частоту
tag_performance_final = pd.merge(tag_performance_sorted, tag_counts, on='tags')

tag_performance_final

,tags,answered_correctly,count
0,68,0.873532,1107
1,187,0.850349,2292
2,149,0.836061,3666
3,130,0.829202,2219
4,70,0.799559,2724
...,...,...,...
183,151,0.482405,5172
184,186,0.473684,76
185,23,0.447502,4543
186,19,0.434585,3088


Также нужно проанализировать датасет с лекциями.
Например, можно узнать влияние категории лекции на правильность ответа на вопросе  

In [47]:
student_lectures_df = train_data[train_data['content_type_id'] == 1]
# Объединить DataFrame'ы по content_id и lecture_id
merged_df = pd.merge(student_lectures_df, lectures_data, 
                     left_on='content_id', right_on='lecture_id', how='inner')

# Группируем данные по 'tag' и вычисляем средний 'answered_correctly'
tag_performance = merged_df.groupby('tag')['answered_correctly'].mean().reset_index()

# Сортируем результаты по убыванию среднего 'answered_correctly'
tag_performance_sorted = tag_performance.sort_values(by='answered_correctly', ascending=False)
tag_performance_sorted



,tag,answered_correctly
0,0,-1.0
94,120,-1.0
96,124,-1.0
97,125,-1.0
98,126,-1.0
...,...,...
50,64,-1.0
51,65,-1.0
52,66,-1.0
53,67,-1.0
